In [1]:
import pathlib
import sys
import os

# Hacky way to get to root package
if "notebooks" in os.getcwd():
    os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
    
import sys
from data_container import load_scouted_data, get_Teams_in_Match, load_pit_data
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.max_columns', None)

In [3]:
scouted_data = load_scouted_data()
scouted_data

,Scouter Initials,Match Number,Team Number,No Show,Auto Fuel,Auto Climbing Status,Auto Human Player Score,Teleop Fuel,Teleop Fuel Passed,Endgame Climbing Level
0,abc,1,5045,True,3,False,6,131,26,L1
1,abc,1,5744,False,14,True,12,49,27,L1
2,abc,1,4630,True,10,True,10,20,12,NaN
3,abc,1,21,False,13,False,6,152,35,L2
4,abc,1,3824,True,9,True,2,175,22,L1
...,...,...,...,...,...,...,...,...,...,...
163,abc,28,4467,False,0,True,14,150,162,NaN
164,abc,28,3843,True,10,False,10,191,22,L3
165,abc,28,3937,True,5,False,9,100,66,L1
166,abc,28,4576,False,14,False,13,181,53,NaN


In [4]:
teams = get_Teams_in_Match()
match_data = scouted_data.loc[scouted_data["Team Number"].isin(teams)].reset_index()
match_data

TypeError: get_Teams_in_Match() missing 1 required positional argument: 'match_number'

In [ ]:
scouted_data = load_scouted_data()
teams = get_Teams_in_Match()
match_data = scouted_data.loc[scouted_data["Team Number"].isin(teams)].reset_index()

# Convert numeric columns, filling NaN with 0
numeric_cols = ["Auto Fuel", "Auto Human Player Score", "Teleop Fuel", "Teleop Human Player Score"]
for col in numeric_cols:
    if col in match_data.columns:
        match_data[col] = pd.to_numeric(match_data[col], errors='coerce').fillna(0)

# Handle boolean column
match_data["Auto Climbing Status"] = match_data["Auto Climbing Status"].fillna(False)
if match_data["Auto Climbing Status"].dtype == 'object':
    match_data["Auto Climbing Status"] = match_data["Auto Climbing Status"].astype(str).str.lower().isin(['true', '1', 'yes'])

# Convert Endgame Climbing Level for teleop
def convert_endgame_to_teleop(level):
    if pd.isna(level):
        return 0
    level_str = str(level).upper().strip()
    if level_str == "L1":
        return 10
    elif level_str == "L2":
        return 20
    elif level_str == "L3":
        return 30
    else:
        return 0

match_data["Endgame Teleop Points"] = match_data["Endgame Climbing Level"].apply(convert_endgame_to_teleop)

# Calculate scores
match_data["Auto Climb Points"] = match_data["Auto Climbing Status"].apply(lambda x: 15 if x else 0)
match_data["All Auto"] = match_data["Auto Fuel"] + match_data["Auto Human Player Score"] + match_data["Auto Climb Points"]
match_data["All Teleop"] = match_data["Teleop Fuel"] + match_data["Teleop Human Player Score"] + match_data["Endgame Teleop Points"]
match_data["All Endgame"] = match_data["Endgame Teleop Points"]
match_data["Auto and Endgame"] = match_data["All Auto"] + match_data["All Endgame"]

# Group by Team Number
team_stats = match_data.groupby("Team Number").agg({
    "All Teleop": "mean",
    "Auto and Endgame": "mean",
    "All Auto": "mean",
    "All Endgame": "mean",
    "Endgame Teleop Points": "mean",
    "Auto Climb Points": "mean",
    "Auto Climbing Status": "mean",
}).reset_index()

# Create plot with Team Number bold at the top
fig = px.scatter(
    team_stats, 
    x="All Teleop", 
    y="Auto and Endgame", 
    title="Teleop vs. Auto + Endgame (Team Averages)",
    hover_data={
        "Team Number": True,
        "All Teleop": ":.1f",
        "Auto and Endgame": ":.1f",
        "All Auto": ":.1f",
        "All Endgame": ":.1f",
        "Endgame Teleop Points": ":.1f",
        "Auto Climb Points": ":.1f",
        "Auto Climbing Status": ":.2%",
    }
)

# Update hover template to make Team Number bold at the top
fig.update_traces(
    hovertemplate=(
        "<b>Team %{customdata[0]}</b><br>"
        "All Teleop: %{x:.1f}<br>"
        "Auto and Endgame: %{y:.1f}<br>"
        "All Auto: %{customdata[1]:.1f}<br>"
        "All Endgame: %{customdata[2]:.1f}<br>"
        "Endgame Teleop Points: %{customdata[3]:.1f}<br>"
        "Auto Climb Points: %{customdata[4]:.1f}<br>"
        "Auto Climbing Status: %{customdata[5]:.2%}<extra></extra>"
    ),
    customdata=team_stats[["Team Number", "All Auto", "All Endgame", 
                          "Endgame Teleop Points", "Auto Climb Points", 
                          "Auto Climbing Status"]].values
)

fig.update_layout(
    xaxis_title="Average Teleop Score",
    yaxis_title="Average (Auto + Endgame) Score",
    hovermode="closest",
    showlegend=False
)

fig.show()